In [327]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import sys

In [303]:
driver = webdriver.Chrome('C:/Users/macle/Downloads/chromedriver_win32 (3)/chromedriver.exe')

driver.get("https://web.whatsapp.com/")
wait = WebDriverWait(driver, 600)

In [417]:
contact = 'Fantasy PL'

In [418]:
group_title = wait.until(EC.presence_of_element_located((By.XPATH, f"//span[@title='{contact}']")))
group_title.click()

In [419]:
message = driver.find_elements_by_xpath('//*[@id="main"]/footer/div[1]/div[2]/div/div[2]')[0]

In [6]:
#message.send_keys(content + Keys.ENTER)

In [2]:
import requests

In [376]:
base = 'https://fantasy.premierleague.com/drf/'

def get_data(player_id):
    
    data = dict(
        picks = base + f'entry/{player_id}/event/38/picks',
        players = base + 'elements/',
        live = base + 'event/38/live',
    )

    return {key: requests.get(url).json() for key, url in data.items()}

In [97]:
class Picks:
    
    def __init__(self, user, picks_data, player_data, live_data):
        self.active_players = [0,1,2,3,4,5,6,7,8,9,10]
        self.user = user
        self.captain = list(filter(lambda x: x['is_captain'] == True, picks_data['picks']))[0]['element']
        self.vice_captain = list(filter(lambda x: x['is_vice_captain'] == True, picks_data['picks']))[0]['element']
        self.picks = self.process_picks(picks_data, player_data, live_data)
        self.points_hit = picks_data['entry_history']['event_transfers_cost']
        
    def process_picks(self, picks_data, player_data, live_data):
        
        picks_dict = [{'name': self.get_player_name(player_data, player['element']),
                       'id': player['element'],
                       'total_points': live_data['elements'][str(player['element'])]['stats']['total_points'] * self.captain_multiplier(player['element'], live_data['elements'][str(self.captain)]['stats']['minutes']),
                       'explain': {key: value['points'] * self.captain_multiplier(player['element'], live_data['elements'][str(self.captain)]['stats']['minutes'])
                                   for key, value in live_data['elements'][str(player['element'])]['explain'][0][0].items()}}
                      for player in picks_data['picks']]
        
        return picks_dict
    
    def get_player_name(self, player_data, player_id):
        return list(filter(lambda x: x['id'] == player_id, player_data))[0]['web_name']
        
    def captain_multiplier(self, player_id, captain_mins):
        if player_id == self.captain:
            return 2
        if player_id == self.vice_captain and captain_mins == 0:
            return 2
        else:
            return 1
        
    @property
    def picks_total_points(self):
        return sum(self.picks[i]['total_points'] for i in self.active_players)

In [391]:
def diff_picks(picks, picks_old):
    
    for pick, pick_old in zip(picks[:11], picks_old[:11]):
        if pick['id'] != pick_old['id']:
            diff = {'name': pick['name'], 'total_points': pick['total_points'], 'explain': pick['explain']}
            yield diff
            diff = {'name': pick_old['name'], 'total_points': -pick_old['total_points']}
            yield diff
            
        diff = {'name': pick['name'], 'total_points': pick['total_points'] - pick_old['total_points']}
        if diff['total_points'] != 0:
            yield diff
        

In [172]:
list(diff_picks(test_obj.picks, test_old_obj.picks))

[{'name': 'Rose', 'total_points': 2, 'explain': {'minutes': 1}}]

In [173]:
test_old_picks = [
    dict(id=0, name='a', total_points=1, explain=dict(dog=1, cat=2)),
    dict(id=1, name='b', total_points=2, explain=dict(dog=1, cat=2)),
    dict(id=2, name='c', total_points=3, explain=dict(dog=1, cat=2)),
    dict(id=3, name='c', total_points=3, explain=dict(dog=1, cat=2)) 
]
test_picks = [
    dict(id=0, name='a', total_points=-1, explain=dict(dog=0, cat=1)),
    dict(id=1, name='b', total_points=7, explain=dict(dog=1, cat=2, horse=5)),
    dict(id=4, name='d', total_points=3, explain=dict(dog=1, cat=2)),
    dict(id=3, name='c', total_points=3, explain=dict(dog=1, cat=2)) 
]

In [174]:
list(diff_picks(test_picks, test_old_picks))

[{'name': 'a', 'total_points': -2, 'explain': {'dog': -1, 'cat': -1}},
 {'name': 'b', 'total_points': 5, 'explain': {'horse': 5}},
 {'name': 'd', 'total_points': 3, 'explain': {'dog': 1, 'cat': 2}},
 {'name': 'c', 'total_points': -3, 'explain': {'dog': -1, 'cat': -2}}]

In [266]:
def diff_message(picks, picks_old):
    output = ''
    for diff in diff_picks(picks.picks, picks_old.picks):
        output += f"{diff['name']}: {diff['total_points']:+}\n"
    return output

In [366]:
def picks_to_message(picks_a, picks_old_a, picks_b, picks_old_b):
    output = '-----------------------------\n'
    output += f'*{picks_a.user} [{picks_a.picks_total_points}]*\n'
    
    diff = diff_message(picks_a, picks_old_a)
    if diff:
        output += diff
        
    output += f'*{picks_b.user} [{picks_b.picks_total_points}]*\n'

    diff = diff_message(picks_b, picks_old_b)
    if diff:
        output += diff
    
    return output
    

In [364]:
print(picks_to_message(test_obj, test_old_obj, test_obj2, test_old_obj2))

-----------------------------
*Matt [42]
*Rose: +2
*Tiago [38]
*Alexander-Arnold: -4
Pogba: +2



In [248]:
def refresh_picks(history, player_ids):
    
    data = {key: get_data(value) for key, value in player_ids.items()}
    picks = {key: Picks(key, value['picks'], value['players'], value['live']) for key, value in data.items()}
    
    for key, value in picks.items():
        history[key].append(value)

START

In [277]:
from collections import defaultdict
import time

In [252]:
ids = {'Tiago': '2308505', 'Matt': '913380'}

In [401]:
test_history = defaultdict(list)

refresh_picks(test_history, ids)
refresh_picks(test_history, ids)

test_history['Matt'][-1].picks[2]['total_points'] = test_history['Matt'][-1].picks[2]['total_points']+2
test_history['Matt'][-1].picks[2]['explain']['minutes'] = test_history['Matt'][-1].picks[2]['explain']['minutes']+1

test_history['Tiago'][-1].picks[3]['total_points'] = test_history['Tiago'][-1].picks[3]['total_points']-4
test_history['Tiago'][-1].picks[3]['explain']['minutes'] = test_history['Tiago'][-1].picks[3]['explain']['minutes']-4
test_history['Tiago'][-1].picks[4]['total_points'] = test_history['Tiago'][-1].picks[4]['total_points']+2
test_history['Tiago'][-1].picks[4]['explain']['minutes'] = test_history['Tiago'][-1].picks[4]['explain']['minutes']+1

In [402]:
tiago_old_picks = test_history['Tiago'][-2]
tiago_picks =  test_history['Tiago'][-1]

matt_old_picks = test_history['Matt'][-2]
matt_picks = test_history['Matt'][-1]

In [406]:
history = defaultdict(list)

In [403]:
while True:
    
    #refresh_picks(history, ids)
    
    tiago_old_picks = test_history['Tiago'][-2]
    tiago_picks =  test_history['Tiago'][-1]
    
    matt_old_picks = test_history['Matt'][-2]
    matt_picks = test_history['Matt'][-1]
    
    if list(diff_picks(tiago_picks.picks, tiago_old_picks.picks)) or list(diff_picks(matt_picks.picks, matt_old_picks.picks)):
        
        content = picks_to_message(tiago_picks, tiago_old_picks, matt_picks, matt_old_picks)
        print('Tiago idx:', len(history['Tiago']))
        print('Matt idx:', len(history['Matt']))
        print(content)
        
    message.send_keys(content + Keys.ENTER)
        
    time.sleep(10)

Tiago idx: 16
Matt idx: 16
-----------------------------
*Tiago [19]*
Davies: -4
Bernardo Silva: +2
*Matt [28]*
Valery: +2



KeyboardInterrupt: 

In [408]:
refresh_picks(history, ids)
refresh_picks(history, ids)

In [ ]:
while True:
    
    refresh_picks(history, ids)
    
    tiago_old_picks = history['Tiago'][-2]
    tiago_picks =  history['Tiago'][-1]
    
    matt_old_picks = history['Matt'][-2]
    matt_picks = history['Matt'][-1]
    
    if list(diff_picks(tiago_picks.picks, tiago_old_picks.picks)) or list(diff_picks(matt_picks.picks, matt_old_picks.picks)):
        
        content = picks_to_message(tiago_picks, tiago_old_picks, matt_picks, matt_old_picks)
        print('Tiago idx:', len(history['Tiago']))
        print('Matt idx:', len(history['Matt']))
        print(content)
        message.send_keys(content + Keys.ENTER)
        
    time.sleep(30)

Tiago idx: 116
Matt idx: 116
-----------------------------
*Tiago [22]*
Fraser: +1
*Matt [31]*

Tiago idx: 118
Matt idx: 118
-----------------------------
*Tiago [37]*
Alexander-Arnold: +5
Davies: +5
Bernardo Silva: +1
Sterling: +1
Agüero: +2
Jiménez: +1
*Matt [41]*
Alexander-Arnold: +5
Mané: +2
Bernardo Silva: +1
Agüero: +2

Tiago idx: 121
Matt idx: 121
-----------------------------
*Tiago [42]*
Leno: +5
*Matt [48]*
Leno: +5
Valery: +1
Redmond: +1

Tiago idx: 131
Matt idx: 131
-----------------------------
*Tiago [39]*
Leno: -3
*Matt [45]*
Leno: -3

Tiago idx: 134
Matt idx: 134
-----------------------------
*Tiago [40]*
Hazard: +1
*Matt [45]*

Tiago idx: 136
Matt idx: 136
-----------------------------
*Tiago [36]*
Davies: -4
*Matt [45]*

Tiago idx: 142
Matt idx: 142
-----------------------------
*Tiago [39]*
Sterling: +3
*Matt [45]*

Tiago idx: 144
Matt idx: 144
-----------------------------
*Tiago [38]*
Davies: -1
*Matt [45]*

Tiago idx: 160
Matt idx: 160
----------------------------